<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/inference_feat/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection ML Test

## 1. Prerequisites

This section provides the setting up of the environment

In [1]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl

Looking in links: https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in links: https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
Looking in links: https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl


In [2]:
!git clone https://github.com/v-iashin/video_features.git
!pip install omegaconf==2.0.6

fatal: destination path 'video_features' already exists and is not an empty directory.


fatal: destination path 'ict3104-p2-team08' already exists and is not an empty directory.


In [3]:
# Imports widgets for jupyter
try:
    import piplite
    piplite.install(['ipywidgets'])
except ImportError:
    pass

## 2. Toggling of pipelines

This section shows the toggling of dependencies between TSU and STEP

In [32]:
#insert code here

C:\Users\leech\ict3104-p2-team08


## 3. Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

In [6]:
#import tsu video files
import os
FOLDER_PATH = 'input_files'
ROOT_PATH = '/content/data'

file_list = os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/data\\input_files'

In [8]:
#tsu video files to display in dropdown
from ipywidgets import  Dropdown, widgets

w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

w.observe(on_change)

display(w)

NameError: name 'file_list' is not defined

In [9]:
#display video 
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/data/input_files/%' + w.value

locate= "" 

for i in video_path: 
   if i=='%': 
       pass
   else: 
       locate+=i 

def show_video(locate, video_width = 500):
  video_file = open(locate, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
show_video(locate)

NameError: name 'w' is not defined

In [ ]:
#only run it for the first time
%cd video_features

In [ ]:
#extract to I3D feature file
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

# Select the feature type
feature_type = 'i3d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/data/input_files/P02T02C06.mp4']
args.flow_type = 'raft'

extractor = ExtractI3D(args)
print(locate)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

## 4. Inference with pretrained model

This section allows generation of caption videos

### Select a pre-trained model

In [153]:
#tsu video files to display in dropdown
import ipywidgets as widgets
import os

MODEL_PATH = 'models'
MODEL_ROOT_PATH = './Toyota_Smarthome/pipline/'

file_list = os.listdir(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))

model_dropdown = widgets.Dropdown(
    options=file_list,
    description='Model:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

model_dropdown.observe(on_change)
display(model_dropdown)

Dropdown(description='Model:', options=('MODELTESST2', 'MODELTEST', 'PDAN'), value='MODELTESST2')

PDAN


### Choose an input video

In [159]:
VIDEO_PATH = 'input_files'
VIDEO_ROOT_PATH = './data/'

full_testable_video = os.listdir(os.path.join(VIDEO_ROOT_PATH, VIDEO_PATH))

model_name = model_dropdown.value
JSON_FILEPATH = "./Toyota_Smarthome/pipline/model_videos.json"
f = open(JSON_FILEPATH)
data = json.load(f)
testable_video = data[model_name]
testable_video = [video + ".mp4" for video in testable_video]
print(testable_video)

if model_name == "PDAN":
    file_list = full_testable_video
else:
    file_list = testable_video

video_dropdown = widgets.Dropdown(
    options=file_list,
    description='Video:',
    disabled=False,
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

video_dropdown.observe(on_change)
display(video_dropdown)

['P02T14C06.mp4', 'P02T15C06.mp4', 'P02T16C06.mp4', 'P02T17C05.mp4', 'P02T18C05.mp4']


Dropdown(description='Video:', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.mp4', 'P…

P02T03C03.mp4


### Generate video with caption

In [160]:
#insert codes here
from IPython.display import Video

video_selected = video_dropdown.value.replace('.mp4','')
model_selected = model_dropdown.value
model_path = "./Toyota_Smarthome/pipline/models/" + model_selected
%run -i ./Toyota_Smarthome/pipline/test2.py -videofile={video_selected} -load_model={model_path} -name={model_selected}
Video("./Toyota_Smarthome/pipline/video_output/" + video_selected + "_caption.mp4")


100%|████████████████████████████████████████████████████████████████████████████████| 536/536 [00:08<00:00, 65.68it/s]

 23%|██████████████████▏                                                            | 123/536 [00:00<00:02, 149.18it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 536/536 [00:03<00:00, 136.46it/s]

generating video...:   5%|██▉                                                         | 38/768 [00:00<00:11, 63.46it/s]


generating video...:  12%|███████▌                                                    | 96/768 [00:01<00:10, 64.24it/s]


generating video...:  20%|███████████▊                                               | 153/768 [00:02<00:09, 63.43it/s]


generating video...:  27%|████████████████▏                                          | 211/768 [00:03<00:08, 63.52it/s]


...

generating video...:  73%|██████████████████████████████████████████▊                | 557/768 [00:08<00:03, 63.49it/s]


generating video...:  80%|███████████████████████████████████████████████▏           | 615/768 [00:09<00:02, 60.43it/s]


generating video...:  88%|███████████████████████████████████████████████████▋       | 672/768 [00:10<00:01, 61.70it/s]


generating video...:  95%|████████████████████████████████████████████████████████   | 730/768 [00:11<00:00, 64.18it/s]


generating video...: 100%|███████████████████████████████████████████████████████████| 768/768 [00:12<00:00, 64.26it/s]
generating video...: : 769it [00:12, 61.70it/s]                                                                        

## 5. Testing and evaluating trained models

This section shows

## 6. Training of new model

This section shows

### Choose datasets for training/testing

In [79]:
#C:\Users\leech\OneDrive\Documents\untrimmed\RGB.tar\Videos_mp4
DATASET_PATH = 'Videos_mp4'
DATASET_ROOT_PATH = '../OneDrive/Documents/untrimmed/RGB.tar/'

file_list = os.listdir(os.path.join(DATASET_ROOT_PATH, DATASET_PATH))

sel = widgets.SelectMultiple(
options=file_list,
#rows=10,
description='Dataset:',
disabled=False
)

selected_videos = []

but = widgets.Button(description = 'Upload')
def on_button_clicked(b):
    print(sel.value)

but.on_click(on_button_clicked)

display(sel)
display(but)

SelectMultiple(description='Dataset:', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.…

Button(description='Upload', style=ButtonStyle())

('P02T15C06.mp4', 'P02T16C06.mp4', 'P02T17C05.mp4', 'P02T18C05.mp4', 'P03T01C05.mp4', 'P03T02C04.mp4')
('P02T14C06.mp4', 'P02T15C06.mp4', 'P02T16C06.mp4', 'P02T17C05.mp4', 'P02T18C05.mp4', 'P03T01C05.mp4', 'P03T02C04.mp4', 'P03T02C05.mp4', 'P03T03C04.mp4')


### Model name, Batch size, Epochs

In [151]:
inp = widgets.Text(description='Model name:')
batch_size = widgets.BoundedIntText(description='Batch size:',
                                   value=2,
                                   max=4)
epochs = widgets.BoundedIntText(description='Epochs:',
                               value=1000,
                               max=1000)
button = widgets.Button(description="Enter")

button = widgets.Button(description="Enter")

def on_button_clicked(b):
    print("Model name set: ", inp.value)
    print("batch size set: ", batch_size.value)
    print("epochs set: ", epochs.value)

button.on_click(on_button_clicked)
display(inp)
display(batch_size)
display(epochs)
display(button)

Text(value='', description='Model name:')

BoundedIntText(value=2, description='Batch size:', max=4)

BoundedIntText(value=1000, description='Epochs:', max=1000)

Button(description='Enter', style=ButtonStyle())

Model name set:  MODELTESST2
batch size set:  1
epochs set:  5


### Start training model

model_name = inp.value
selected_videos = list(sel.value)
for i in range(len(selected_videos)):
    selected_videos[i] = selected_videos[i].replace('.mp4','')
selected_videos_string=(','.join(selected_videos))
batchSize = str(batch_size.value)
epochValue = (epochs.value)
%run -i ./Toyota_Smarthome/pipline/train.py -video_train_test={selected_videos_string} -model=PDAN -name={model_name} -batch_size={batchSize} -epoch={epochValue}